In [1]:
import pandas as pd 
import numpy as np
from datetime import datetime, timedelta
# from pandas_profiling import ProfileReport

In [2]:
data = pd.read_csv("data/deliverytime.csv")


In [20]:
import math

def calculate_distance(lat1, lon1, lat2, lon2):
    # Convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    radius_of_earth = 6371  # in kilometers
    distance = radius_of_earth * c

    return round(distance,2)

In [22]:
data['distace']=data.apply(lambda row: calculate_distance(row['Restaurant_latitude'],
                                                             row['Restaurant_longitude'],
                                                             row['Delivery_location_latitude'],
                                                             row['Delivery_location_longitude']), axis=1)

In [ ]:

data['Time_Orderd']=pd.DatetimeIndex(data["Order_Date"])
# data['orderd_hour'] = pd.to_datetime(data['Time_Orderd'], format='%H:%M').dt.hour

data.info()

In [ ]:
def convert_datetime(data,col_name):
    return pd.DatetimeIndex(data[col_name])
    

In [ ]:
convert_datetime(data,"Order_Date")

In [6]:
data['Time_taken (min)']% 60

0        46
1        23
2        21
3        20
4        41
         ..
45579    32
45580    36
45581    16
45582    26
45583    36
Name: Time_taken (min), Length: 45584, dtype: int64

In [ ]:
data["Order_Date"]=pd.to_datetime(data["Order_Date"],infer_datetime_format=True)

In [ ]:
data["Ordered_Year"]=data["Order_Date"].dt.year
data["Ordered_Month"]=data["Order_Date"].dt.month
data["Ordered_Day"]=data["Order_Date"].dt.day 


In [ ]:
data=data.drop(columns="Order_Date",axis=1)

In [ ]:
data["Time_Orderd"]=data["Time_Orderd"].astype('str')
data["Time_Orderd"]=data["Time_Orderd"].apply(lambda x : x[0] if "." in x else x)
data["Time_Orderd"]=data["Time_Orderd"].apply(lambda x :'0'+x+":00" if x=='1' else x)
data["Time_Orderd"]=data["Time_Orderd"].apply(lambda x :'01'+x[2:5] if x[0:2]=='24' else x)
data["Time_Orderd"]=data["Time_Orderd"].apply(lambda x :'0' if x=='nan'else x)
data["Time_Orderd"]=data["Time_Orderd"].apply(lambda x :'00:00' if x=='0'else x)
data['Time_Orderd'] = np.where(data['Time_Orderd'] == '00:00', data.apply(subtract_time, axis=1), data['Time_Orderd'])


data["Time_Order_picked"]=data["Time_Order_picked"].astype('str')
data["Time_Order_picked"]=data["Time_Order_picked"].apply(lambda x : x[0] if "." in x else x)
data["Time_Order_picked"]=data["Time_Order_picked"].apply(lambda x :'0'+x+":00" if x=='1' else x)
data["Time_Order_picked"]=data["Time_Order_picked"].apply(lambda x :'01'+ x[2:5] if x[0:2]=='24' else x)
data["Time_Order_picked"]=data["Time_Order_picked"].apply(lambda x :'0' if x=='nan'else x)
data["Time_Order_picked"]=data["Time_Order_picked"].apply(lambda x :'00:00' if x=='0'else x)
data['Time_Order_picked'] = np.where(data['Time_Order_picked'] == '00:00', data.apply(add_time, axis=1), data['Time_Order_picked'])



In [ ]:
def subtract_time(row):
    if row['Time_Orderd'] == '00:00':
        pickup_time = datetime.strptime(row['Time_Order_picked'], "%H:%M")
        time_diff = timedelta(hours=0, minutes=15)
        updated_time = pickup_time - time_diff
        return updated_time.strftime("%H:%M")
    else:
        return row['Time_Orderd']


In [ ]:
def add_time(row):
    if row['Time_Order_picked'] == '00:00':
        orderd_time = datetime.strptime(row['Time_Orderd'], "%H:%M")
        time_diff = timedelta(hours=0, minutes=15)
        updated_time = orderd_time + time_diff
        return updated_time.strftime("%H:%M")
    else:
        return row['Time_Order_picked']

In [ ]:
data.info()

In [ ]:
data['Delivery_person_Age']=data['Delivery_person_Age'].astype('int')

In [ ]:
data[data['Delivery_person_Ratings'].isnull()]

In [ ]:
data["orderd_hour"]=data["Time_Orderd"].apply(lambda x : x[0:2])

In [ ]:
data["orderd_minute"]=data["Time_Orderd"].apply(lambda x : x[3:])

In [ ]:
data=data.drop(columns="Time_Orderd",axis=1)

In [ ]:
data["orderd_picked_hour"]=data["Time_Order_picked"].apply(lambda x : x[0:2])
data["orderd_picked_minute"]=data["Time_Order_picked"].apply(lambda x : x[3:])

In [ ]:
data=data.drop(columns="Time_Order_picked",axis=1)

In [7]:
data.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min)
0,0xcdcd,DEHRES17DEL01,36.0,4.2,30.327968,78.046106,30.397968,78.116106,12-02-2022,21:55,22:10,Fog,Jam,2,Snack,motorcycle,3.0,No,Metropolitian,46
1,0xd987,KOCRES16DEL01,21.0,4.7,10.003064,76.307589,10.043064,76.347589,13-02-2022,14:55,15:05,Stormy,High,1,Meal,motorcycle,1.0,No,Metropolitian,23
2,0x2784,PUNERES13DEL03,23.0,4.7,18.562450,73.916619,18.652450,74.006619,04-03-2022,17:30,17:40,Sandstorms,Medium,1,Drinks,scooter,1.0,No,Metropolitian,21
3,0xc8b6,LUDHRES15DEL02,34.0,4.3,30.899584,75.809346,30.919584,75.829346,13-02-2022,09:20,09:30,Sandstorms,Low,0,Buffet,motorcycle,0.0,No,Metropolitian,20
4,0xdb64,KNPRES14DEL02,24.0,4.7,26.463504,80.372929,26.593504,80.502929,14-02-2022,19:50,20:05,Fog,Jam,1,Snack,scooter,1.0,No,Metropolitian,41


In [ ]:
data["orderd_hour"]=data["orderd_hour"].astype(int)
data["orderd_minute"]=data["orderd_minute"].astype(int)
data["orderd_picked_hour"]=data["orderd_picked_hour"].astype(int)
data["orderd_picked_minute"]=data["orderd_picked_minute"].astype(int)

In [ ]:
data.info()

In [ ]:
numerical=[]
categorical=[]
for i in data.columns:
    if data[i].dtype=="object":
        categorical.append(i)
    else:
        numerical.append(i)

In [ ]:
numerical

In [ ]:
categorical

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Generate a regression dataset
X, y = make_regression(n_samples=1000, n_features=10, random_state=42)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create Random Forest and Gradient Boosting models
rf_model = RandomForestRegressor(n_estimators=100)
gb_model = GradientBoostingRegressor(n_estimators=100)

# Fit the models on the training data
rf_model.fit(X_train, y_train)
gb_model.fit(X_train, y_train)

# Make predictions using the ensemble method
rf_predictions = rf_model.predict(X_test)
gb_predictions = gb_model.predict(X_test)

# Combine the predictions using averaging
ensemble_predictions = (rf_predictions + gb_predictions) / 2

# Calculate the mean squared error for each model and the ensemble
rf_mse = mean_squared_error(y_test, rf_predictions)
gb_mse = mean_squared_error(y_test, gb_predictions)
ensemble_mse = mean_squared_error(y_test, ensemble_predictions)

print("Random Forest MSE:", rf_mse)
print("Gradient Boosting MSE:", gb_mse)
print("Ensemble MSE:", ensemble_mse)
